In [ ]:
# !pip install tensorflow==1.15.0

# !pip install tensorflow-gpu==1.15.0

In [2]:
import sys
sys.path.append("..")
import pickle
import interpret_utils
import matplotlib.pyplot as plt
from scziDesk_preprocess import *
from scziDesk_network import *
from scziDesk_utils import *
import argparse
import h5py
import time
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score, silhouette_score, calinski_harabasz_score
from collections import Counter
import glob2
plt.ion()
plt.show()
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
random_seed = [1111, 2222, 3333, 4444, 5555, 6666, 7777, 8888, 9999, 10000]

parser = argparse.ArgumentParser(description="train", formatter_class=argparse.ArgumentDefaultsHelpFormatter)
# parser.add_argument("--dataname", default = "Quake_10x_Bladder", type = str)
parser.add_argument("--dataname", default = "data_-1.5c15", type = str)

parser.add_argument("--distribution", default = "ZINB")
parser.add_argument("--self_training", default = True)
parser.add_argument("--dims", default = [499, 256, 64, 32])
parser.add_argument("--highly_genes", default = 500)
parser.add_argument("--alpha", default = 0.001, type = float)
parser.add_argument("--gamma", default = 0.001, type = float)
parser.add_argument("--learning_rate", default = 0.0001, type = float)
parser.add_argument("--random_seed", default = random_seed)
parser.add_argument("--batch_size", default = 256, type = int)
parser.add_argument("--update_epoch", default = 10, type = int)
parser.add_argument("--pretrain_epoch", default = 1000, type = int)
parser.add_argument("--funetrain_epoch", default = 2000, type = int)
parser.add_argument("--t_alpha", default = 1.0)
parser.add_argument("--noise_sd", default = 1.5)
parser.add_argument("--error", default = 0.001, type = float)
parser.add_argument("--gpu_option", default = "0")

args = parser.parse_args("")

In [4]:
method = "scziDesk"
nb_features = -1
pval_cutoff=None
run = 0
for category in [ "imbalanced_data",  "balanced_data"
                ]:

    path= ".."
    if category in ["imbalanced_data", "balanced_data"]:
        files = glob2.glob(f'{path}/R/simulated_data/{category}/*.h5')
        files = [f[len(f"{path}/R/simulated_data/{category}/"):-3] for f in files][::-1]
    else:
        files = glob2.glob(f'{path}/real_data/*.h5')
        files = [f[len(f"{path}/real_data/"):-3] for f in files]
    print(files)


    df = pd.DataFrame(columns = ["dataset", "ARI", "NMI", "sil", "run", "time", "pred", "cal", "features"])
    for dataset in files:
        if category in ["imbalanced_data", "balanced_data"]:
            data_mat = h5py.File(f"{path}/R/simulated_data/{category}/{dataset}.h5","r")
        else:
            data_mat = h5py.File(f"{path}/real_data/{dataset}.h5","r")

        Y = np.array(data_mat['Y'])
        X = np.array(data_mat['X'])
        print(f">>>>dataset {dataset}")
        if X.shape[0] > 10000:
            continue

        X = np.ceil(X).astype(np.int)
        count_X = X
        print(X.shape, count_X.shape)
        orig_X = X.copy()
        adata = sc.AnnData(X)
        adata.obs['Group'] = Y
        adata = normalize(adata,
                          copy=True,
                          highly_genes=args.highly_genes,
                          size_factors=True,
                          normalize_input=True,
                          logtrans_input=True)
        X = adata.X.astype(np.float32)
        Y = np.array(adata.obs["Group"])

        high_variable = np.array(adata.var.highly_variable.index, dtype=np.int)
        count_X = count_X[:, high_variable]
        size_factor = np.array(adata.obs.size_factors).reshape(-1,
                                                               1).astype(np.float32)
        cluster_number = int(max(Y) - min(Y) + 1)
        args.dims[0]=X.shape[1]
        print(X.shape, count_X.shape)
        idx = adata.var_names.astype(int).tolist()

        start = time.time()
        seed = run
        np.random.seed(seed)
        tf.reset_default_graph()
        chencluster = autoencoder(args.dataname, args.distribution,
                                  args.self_training, args.dims, cluster_number,
                                  args.t_alpha, args.alpha, args.gamma,
                                  args.learning_rate, args.noise_sd)

        chencluster.pretrain(X, count_X, size_factor, args.batch_size,
                             args.pretrain_epoch, args.gpu_option)

        chencluster.funetrain(X, count_X, size_factor, args.batch_size,
                              args.funetrain_epoch, args.update_epoch, args.error, Y)


        clusters = chencluster.Y_pred

        interpret_utils.de_analysis([X,  np.array(data_mat['X'])],
                                    ["proc_", "full_"],
                                    data_mat,
                                    idx,
                                    method,
                                    dataset,
                                    category,
                                    clusters,
                                    nb_features=nb_features,
                                    run=run,
                                    pval_cutoff=pval_cutoff)
        folder = f"../output/interpretability/{category}/{method}"
        write_to = f"{folder}/{dataset}"

        start = time.time()
        with open(f"{write_to}_all.pkl", 'rb') as f:
            results= pickle.load(f)
        evaluated_gradients = chencluster.gradients

        results["features"]["saliency"] = []
        results["features"]["grad_x_input"] = []
        stot = pd.DataFrame()
        grad_time = 0
        for c in np.sort(np.unique(results["meta"]["clusters"])):
            ii = np.where(results["meta"]["clusters"] ==c)[0]
        #     g1 = evaluated_gradients[ii] * adata.X[ii]
        #     g1 = g1.mean(axis = 0)
            g1 = evaluated_gradients[ii].mean(axis = 0)
            t1 = time.time()
            grad_x_input = evaluated_gradients[ii] * adata.X[ii]
            grad_x_input = grad_x_input.mean(0)
            t2 = time.time()
            grad_time += (t2-t1)
            saliency = np.abs(g1)
        #     saliency = g1

            s_r= np.argsort(saliency)[::-1][:nb_features].astype(str)
            results["features"]["saliency"].append(s_r)
            s = pd.DataFrame()
            s["x"] = s_r
            s["saliency"] = np.sort(saliency)[::-1][:nb_features]
            s["cluster"] = c
#             print(">> saliency ", len(np.intersect1d(results["features"]["truth"][int(c)], s_r)))
            t1 = time.time()
            gi_r= np.argsort(grad_x_input)[::-1][:nb_features].astype(str)
            results["features"]["grad_x_input"].append(gi_r)
            gi = pd.DataFrame()
            gi["x"] = gi_r
            gi["grad_x_input"] = np.sort(grad_x_input)[::-1][:nb_features]
            gi["cluster"] = c
            t2 =time.time()
            grad_time += (t2-t1)
        end = time.time()
        results["time"][f"saliency"] = end - start - grad_time
        results["time"][f"grad_x_input"] = end - start
#             print(">> gi ", len(np.intersect1d(results["features"]["truth"][int(c)], gi_r)))
        s["rank"] = s.groupby("cluster")["saliency"].rank("dense", ascending=False)
        gi["rank"] = gi.groupby("cluster")["grad_x_input"].rank("dense", ascending=False)
        results["scores"]["saliency"] = s
        results["scores"]["grad_x_input"] = gi
        with open(f"{write_to}_all.pkl", 'wb') as f:
            pickle.dump(results, f, pickle.HIGHEST_PROTOCOL)

['data_0c_2de_0.3', 'data_-1c_2de_0.05', 'data_0c_2de_0.1', 'data_-1c_3de_0.1', 'data_0c_3de_0.05', 'data_-1c_2de_0.1', 'data_1c_2de_0.1', 'data_0c_3de_0.3', 'data_1c_3de_0.1', 'data_-1c_2de_0.3', 'data_1c_2de_0.05', 'data_-1c_3de_0.3', 'data_1c_2de_0.3', 'data_1c_3de_0.05', 'data_-1c_3de_0.05', 'data_0c_2de_0.05', 'data_0c_3de_0.1', 'data_1c_3de_0.3']
>>>>dataset data_0c_2de_0.3
(3000, 2500) (3000, 2500)
(3000, 500) (3000, 500)


Instructions for updating:
If using Keras pass *_constraint arguments to layers.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


begin the pretraining





begin the funetraining
ARI 1.0, NMI 1.0
0 ARI 1.0, NMI 1.0
ARI before: 1.0
Max nb_features 181
ARI after: 1.0


invalid value encountered in log2
invalid value encountered in log2
The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].


[17:33:53] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:34:35] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


ntree_limit is deprecated, use `iteration_range` or model slicing instead.
overflow encountered in expm1
invalid value encountered in true_divide


feature_importances (3000, 500)


...

ntree_limit is deprecated, use `iteration_range` or model slicing instead.


feature_importances (3000, 499)


overflow encountered in expm1
invalid value encountered in true_divide
overflow encountered in expm1
invalid value encountered in true_divide


[18:54:49] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:09:04] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
feature_importances (3000, 2500)
>>>>dataset data_-1c_2de_0.1
(3000, 2500) (3000, 2500)
(3000, 500) (3000, 500)
begin the pretraining


KeyboardInterrupt: 